In [2]:
import numpy as np
import pandas as pd
import random
import math

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/OHrydko/Machine-Learning-Labs/master/lab3/MP-07-Hrydko.csv', 
                 header=None, names=['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6','x7', 'x8', 'x9', 'x10', 'x11', 'y_s'], sep=';',engine='python')

df.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,y_s
0,x,y,n,t,a,c,b,n,e,y,n,p,edible
1,b,s,y,t,l,c,b,k,e,s,n,g,edible
2,f,s,w,t,a,w,n,n,t,s,u,d,edible
3,b,y,y,t,l,c,b,w,e,s,k,g,edible
4,b,y,y,t,a,c,b,k,e,s,n,m,edible


In [4]:
df = df.replace({'edible': 1, 'poisonous': 0})
df = pd.get_dummies(df)
df['y'] = df['y_s']
df = df.drop(['y_s'], axis=1)
dataset = []
for index, row in df.iterrows():
    dataset.append(list(row))
df.head()


,x0_b,x0_f,x0_s,x0_x,x1_f,x1_s,x1_y,x2_e,x2_g,x2_n,...,x9_y,x10_k,x10_n,x10_u,x11_d,x11_g,x11_m,x11_p,x11_u,y
0,0,0,0,1,0,0,1,0,0,1,...,1,0,1,0,0,0,0,1,0,1
1,1,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
2,0,1,0,0,0,1,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
3,1,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,1
4,1,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,1


In [6]:
def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

def mean(numbers):
    return sum(numbers)/float(len(numbers))

def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

def calculateProbability(x, mean, stdev):
#     print(x, mean, stdev)
    if stdev > 0: 
        exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
        return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent
    else: 
        return 0.5 
    
def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
#     print(probabilities)
    return probabilities
            
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

def getAccuracy(testSet, predictions):
    correct = 0
    testY = []
    for i in range(len(testSet)):
        testY.append(testSet[i][-1])
        if testSet[i][-1] == predictions[i]:
            correct += 1
    print('test data = {}'.format(testY))
    print('prediction data = {}'.format(predictions))
    return (correct/float(len(testSet))) * 100.0

def main():
    splitRatio = 0.9
    trainingSet, testSet = splitDataset(dataset, splitRatio)
    print('Split {0} rows into train={1} and test={2} rows'.format(len(dataset), len(trainingSet), len(testSet)))
    # prepare model
    summaries = summarizeByClass(trainingSet)
#     print(summaries)
#     print(summaries)
    # test model
    predictions = getPredictions(summaries, testSet)
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy: {0}%'.format(accuracy))

main()

Split 250 rows into train=225 and test=25 rows
test data = [1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
prediction data = [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]
Accuracy: 84.0%


In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/OHrydko/Machine-Learning-Labs/master/lab3/MP-07-Hrydko.csv', 
                 header=None, names=['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6','x7', 'x8', 'x9', 'x10', 'x11', 'y_s'], sep=';',engine='python')

df.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,y_s
0,x,y,n,t,a,c,b,n,e,y,n,p,edible
1,b,s,y,t,l,c,b,k,e,s,n,g,edible
2,f,s,w,t,a,w,n,n,t,s,u,d,edible
3,b,y,y,t,l,c,b,w,e,s,k,g,edible
4,b,y,y,t,a,c,b,k,e,s,n,m,edible


In [ ]:
train, test = df[:200], df[:200]
